In [1]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
import panel as pn
import panel.widgets as pnw
import random

In [2]:
def kmtoNaut(km):
    return km / 1.852

In [3]:
N = ['SHA', 'ROT', 'CAP', 'SUE', 'PSA']

In [4]:
distances = [
    ['CAP', 'ROT', 11367.07658],
    ['CAP', 'SUE', 9814.514498],
    ['CAP', 'SHA', 14090.03741],
    ['ROT', 'PSA', 6013.579478],
    ['PSA', 'SUE', 162.274111],
    ['SHA', 'SUE', 13162.53259]]

In [5]:
for i in range(len(distances)):
    distances[i][2] = kmtoNaut(distances[i][2])

In [10]:
class ShippingNetwork(Model):
    def __init__(self, N, distances, S=2):
        self.ports = N
        self.num_ships = S
        # self.distances = distances
        self.schedule = SimultaneousActivation(self)

        self.G = nx.Graph()
        self.G.add_nodes_from(self.ports) #instatiate the ports as nodes of network
        for i in range(len(distances)): #create bi-directional edges with an attribute for length 
            self.G.add_edge(distances[i][0], distances[i][1], length=distances[i][2])
        self.grid = NetworkGrid(self.G) #Define Mesa Grid as the just created Network to allow for shipping only in routes

        #create agents 
        Ships = []
        for i in range(self.num_ships):
            port = random.sample(self.ports, k=2) # this is a test - implementation, more complex algorithming is necessary
            a = Ship(i+1, self, self.G, port[0], port[1], distances)
            #append to list of ships
            Ships.append(a)
            #place agent on origin node
            self.grid.place_agent(a, port[0])
        
    #create ability to remove edges mid-model
    def network_change (self, change_type, change_edge):
        if change_type == "add":
            self.G.add_edge(self.change_edge[0], self.change_edge[1], length=self.change_edge[2])
        if change_type == "remove":
            self.G.remove_edge(self.change_edge[0],self.change_edge[1])
        #update model with new grid    
        return NetworkGrid(self.G), G

    def step(self, distances, change_type='', change_edge=[]):
        #check network for changes
        if change_type != '':
            self.grid, self.G = network_change(change_type, change_edge)
        #Run each Agents
        self.schedule.step()

        # draw the network? 



In [7]:
class Ship(Agent):
    def __init__(self, unique_id, model, G, start_port, destin_port, distances, s=13):
        self.destination = destin_port
        self.state = 0 #0 for active, numbers > 0 for weeks that ships have to "wait" until arrival to port
        self.speed = s*24*7 #speed is given in knots, with 1 knot being 1 nautical mile per hour
        self.position = start_port
        self.init_route = nx.dijkstra_path(G, self.position, self.destination, weight='length') #We keep a copy of the entire itinerary / distance traveled
        self.init_dist = nx.dijkstra_path_length(G,self.position, self.destination, weight='length') 
        self.current_route, self.current_dist = self.init_route, self.init_dist  #For comparison & navigational purposes, we use current route & distance
        self.next_position = self.current_route[1]
        self.itinerary = [self.position] # we keep track of the distance traveled using 
        self.distance_traveled = 0


    def routing(self, G):
        #implement dijkstra to define shortest route
        route = nx.dijkstra_path(G, self.position, self.destination, weight='length')
        travel_distance = nx.dijkstra_path_length(G,self.position, self.destination, weight='length')
        #include a way for capacity?
        return route, travel_distance
    
    def move(self, distances):
        self.step_size = ident_distance(distances) #look up the distance between two cities 
        self.state = step_size / self.speed #change state to step amount
        self.current_dist = self.current_dist - self.step_size #adjust current distance minus the distance traveled in the next step
        self.model.grid.move_agent(self, next_position) #move the agent
        self.current_route.remove(current_route[0]) #remove the next step from the itinerary
        self.next_position = self.current_route[1] #update current route


    def ident_distance(self, distances): #look up the distance of the current step
        for i in range(len(distances)):
            if distances[i][0] == self.position and distances[i][1] == self.next_position:
                dist = distances[i][2]
                return dist
            if distances[i][1] == self.next_position and distances[i][0] == self.position:
                dist = distances[i][2]
                return dist

    def step(self, distances, G):
        self.state = self.state - 1 #'move' ships by one weeks progress
        if self.state <= 0: #ships that are en-route to the node they are going to next do not move / perform other activities
            self.itinerary.append(self.position) #add the current position to itinerary
            new_route, new_distance = routing(G) #perform a new routing to compare against current routing
            if new_route == self.current_route: #if current routing is the same as new, just move (default case)
                self.move(distances)
            elif new_distance > self.current_dist: #if current route is shorter than newly calculated route, check for obstructions
                for i in range(len(self.current_route)-1):
                    if not has_path(G, current_route[i], current_route[i+1]): 
                        self.current_route = new_route
                        self.current_dist = new_distance
                self.move(distances)
            else: # final option is that current route is longer than new route (think Suez reopening after a while), here, we just take the new option
                self.current_route = new_route
                self.current_dist = new_distance
                self.move(distances)


    
    # def collect_time

    # def collect_costs

In [11]:
model = ShippingNetwork(N, distances)
steps = 2
for i in range(steps):
    model.step(distances)